In [12]:
import numpy as np
import pandas as pd
data = pd.read_csv("amazon_baby.csv")
#pd.options.display.max_rows=10

In [13]:
data

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5


In [14]:
def remove_punctuation(text):
    import string
    return text.translate(str.maketrans('','',string.punctuation))

In [15]:
data['review'] = data['review'].fillna('')
data['review_clean'] = data['review'].apply(remove_punctuation)

In [17]:
#2) Extract Sentiments
#remove all 3* reviews
data = data[data['rating'] != 3]

data['sentiment'] = data['rating'].apply(lambda x : +1 if x > 3 else -1)

In [18]:
pd.options.display.max_rows=10
data['sentiment']

1         1
2         1
3         1
4         1
5         1
         ..
183526    1
183527    1
183528    1
183529    1
183530    1
Name: sentiment, Length: 166752, dtype: int64

In [19]:
#3) Split into training and test sets
#vtrain_data,test_data = data.random_split(.8, seed=0)
#train_data, test_data = np.split(data.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
msk = np.random.rand(len(data)) < 0.8
train_data = data[msk]
test_data = data[~msk]

In [20]:
#4) Build the word count vector for each review
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern = r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
test_matrix = vectorizer.fit_transform(test_data['review_clean'])

In [21]:
#5) Train a sentiment classifier with logistic regression
from sklearn.linear_model import LogisticRegression
logisticRegression = LogisticRegression()
sentiment_model = logisticRegression.fit(train_matrix, y=train_data["sentiment"])

In [83]:
# 5 c.d. + 6) Prediciting Sentiment
y = []
counter = 0
arrSize = sentiment_model.coef_.size
for i in range(arrSize):
    if sentiment_model.coef_.flat[i] >= 0:
        counter += 1
        y.append(+1)
    else:
        y.append(-1)
print("Number of positive reviews: ", counter)

Number of positive reviews:  85472


In [99]:
a = sentiment_model.predict(train_matrix)
print(a)

[1 1 1 ..., 1 1 1]


In [106]:
# 7) Probability Predictions
P = []
arrSize = sentiment_model.coef_.size
for i in range(arrSize):
    P.append(1 / (1 + np.exp(-sentiment_model.coef_.flat[i])))

In [ ]:
# 8) 